# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [4]:
class Matrix:
    def __init__(self,*args):
        if len(args)==2:
            # Create Matrix size n x m with zeros
            self.n,self.m=args
            self.matrix=[[0.0 for _ in range(self.m)] for _ in range(self.n)]
        elif len(args)==1 and isinstance(args[0],list):
            # Or initialize with a list of lists
            self.matrix=args[0]
            self.n=len(self.matrix)
            self.m=len(self.matrix[0]) if self.n>0 else 0
            # Check if all rows have the same number of columns
            if not all(len(row)==self.m for row in self.matrix):
                raise ValueError("All rows must have the same number of columns")
        else:
            raise ValueError("Invalid arguments to initialize Matrix")

    def shape(self):
        return (self.n,self.m)

    def transpose(self):
        return Matrix([[self.matrix[j][i] for j in range(self.n)] for i in range(self.m)])

    def row(self, n):
        if n<0 or n>=self.n:
            raise IndexError("Row index out of range")
        return Matrix([self.matrix[n]])

    def column(self, n):
        if n<0 or n>=self.m:
            raise IndexError("Column index out of range")
        return Matrix([[self.matrix[i][n]] for i in range(self.n)])

    def to_list(self):
        return self.matrix

    def block(self,n_0,n_1,m_0,m_1):
        if n_0<0 or n_1>=self.m or m_0<0 or m_1>=self.n:
            raise IndexError("Block indices out of range")
        return Matrix([row[n_0:n_1+1] for row in self.matrix[m_0:m_1+1]])

    def __getitem__(self,idx):
        if isinstance(idx,tuple):
            i,j=idx
            return self.matrix[i][j]
        elif isinstance(idx,slice):
            return [row[idx] for row in self.matrix]
        else:
            return self.matrix[idx]

    def __setitem__(self,idx,value):
        if isinstance(idx,tuple):
            i,j=idx
            self.matrix[i][j]=value
        elif isinstance(idx,slice):
            self.matrix[idx]=value
        else:
            self.matrix[idx]=value

    def scalarmul(self,c):
        return Matrix([[c*self.matrix[i][j] for j in range(self.m)] for i in range(self.n)])

    def add(self,N):
        if self.n !=N.n or self.m !=N.m:
            raise ValueError("Matrices must be the same size to add")
        return Matrix([[self.matrix[i][j]+N.matrix[i][j] for j in range(self.m)] for i in range(self.n)])

    def sub(self,N):
        if self.n !=N.n or self.m !=N.m:
            raise ValueError("Matrices must be the same size to subtract")
        return Matrix([[self.matrix[i][j]-N.matrix[i][j] for j in range(self.m)] for i in range(self.n)])

    def mat_mult(self,N):
        if self.m !=N.n:
            raise ValueError("Incompatible dimensions for matrix multiplication")
        result = [[0.0 for _ in range(N.m)] for _ in range(self.n)]
        for i in range(self.n):
            for j in range(N.m):
                result[i][j]=sum(self.matrix[i][k]*N.matrix[k][j] for k in range(self.m))
        return Matrix(result)

    def element_mult(self,N):
        if self.n !=N.n or self.m !=N.m:
            raise ValueError("Matrices must be the same size for element-wise multiplication")
        return Matrix([[self.matrix[i][j]*N.matrix[i][j] for j in range(self.m)] for i in range(self.n)])

    def equals(self,N):
        return self.n==N.n and self.m==N.m and all(self.matrix[i][j]==N.matrix[i][j] for i in range(self.n) for j in range(self.m))

    # Overload operators for matrix operations
    def __mul__(self,other):
        if isinstance(other, Matrix):
            return self.mat_mult(other)
        elif isinstance(other,(int,float)):
            return self.scalarmul(other)
        else:
            raise TypeError("Unsupported multiplication")

    def __rmul__(self,other):
        if isinstance(other,(int,float)):
            return self.scalarmul(other)
        else:
            raise TypeError("Unsupported multiplication")

    def __add__(self,other):
        if isinstance(other,Matrix):
            return self.add(other)
        else:
            raise TypeError("Unsupported addition")

    def __sub__(self,other):
        if isinstance(other,Matrix):
            return self.sub(other)
        else:
            raise TypeError("Unsupported subtraction")

    def __eq__(self,other):
        return self.equals(other)

    def __repr__(self):
        return f"Matrix({self.matrix})"


# Matrix functions (not methods of Matrix class)
def constant(n,m,c):
    return Matrix([[c for _ in range(m)] for _ in range(n)])

def zeros(n,m):
    return Matrix([[0.0 for _ in range(m)] for _ in range(n)])

def ones(n,m):
    return Matrix([[1.0 for _ in range(m)] for _ in range(n)])

def eye(n):
    return Matrix([[1.0 if i==j else 0.0 for j in range(n)] for i in range(n)])


# Tests
if __name__=="__main__":
    # Initialize some matrices
    A=Matrix([[1,2],[3,4]])
    B=Matrix([[5,6],[7,8]])
    C=Matrix([[9,10],[11,12]])

    print("Matrix A:",A)
    print("Matrix B:",B)
    print("Matrix C:",C)
    print("\n")
    
    # Addition
    D=A+B
    print(f"A+B = {D}")

    # Multiplication
    E=2*A
    print(f"2*A = {E}")

    # Matrix multiplication
    F=A*B
    print(f"A*B = {F}")

    # Transpose
    G=A.transpose()
    print(f"Transpose of A = {G}")

    # block of the matrix
    block=A.block(0,1,0,1)
    print(f"Block of A = {block}")

    # Check identity matrix
    I=eye(2)
    print(f"Identity matrix: {I}")

    # Scalar multiplication
    H=A.scalarmul(3)
    print(f"A*3 = {H}")

    # Matrix equality
    print(f"A equals B: {A == B}")
    print(f"A equals A: {A == A}")

    # Test some edge cases (like dimension mismatch)
    try:
        A+C
    except ValueError as e:
        print(e)

Matrix A: Matrix([[1, 2], [3, 4]])
Matrix B: Matrix([[5, 6], [7, 8]])
Matrix C: Matrix([[9, 10], [11, 12]])


A+B = Matrix([[6, 8], [10, 12]])
2*A = Matrix([[2, 4], [6, 8]])
A*B = Matrix([[19, 22], [43, 50]])
Transpose of A = Matrix([[1, 3], [2, 4]])
Block of A = Matrix([[1, 2], [3, 4]])
Identity matrix: Matrix([[1.0, 0.0], [0.0, 1.0]])
A*3 = Matrix([[3, 6], [9, 12]])
A equals B: False
A equals A: True
